In [2]:
import pandas as pd
#df = pd.read_csv("testbed1.csv")

In [3]:
import spacy
from spacy_langdetect import LanguageDetector
from spacy.language import Language
nlp = spacy.load('en_core_web_lg')
@Language.factory('language_detector')
def language_detector(nlp, name):
    return LanguageDetector()
nlp.add_pipe('language_detector')

In [3]:
data = df['full_tweet_text'].to_list()

In [4]:
isEnglish = []
for text in data:
    doc = nlp(text)
    if doc._.language['score']>0.95:
        isEnglish.append(True)
    else:
        isEnglish.append(False)

In [6]:
df['iseng'] = isEnglish

In [8]:
df = df[df['iseng'] == True]

In [78]:
import sys
from allTokens import *
import os
import time
import pandas as pd
from langdetect import detect

extraChar = {'&quot;': '"',
 '&amp;': 'and',
 '&lt;': '<',
 '&gt;': '>',
 '&nbsp;': 'un-linebreak-able space',
 '&iexcl;': '¡',
 '&cent;': '¢',
 '&pound;': '£',
 '&curren;': '¤',
 '&yen;': '¥',
 '&brvbar;': '¦',
 '&sect;': '§',
 '&uml;': '¨',
 '&copy;': '©',
 '&ordf;': 'ª',
 '&laquo;': '«',
 '&not;': '¬',
 '&shy;': '\xad',
 '&reg;': '®',
 '&macr;': '¯',
 '&deg;': '°',
 '&plusmn;': '±',
 '&sup2': '²',
 '&sup3;': '³',
 '&acute;': '´',
 '&micro;': 'µ',
 '&para;': '¶',
 '&middot;': '·',
 '&cedil;': '¸',
 '&sup1;': '¹',
 '&ordm;': 'º',
 '&raquo;': '»',
 '&frac14;': '¼',
 '&frac12;': '½',
 '&frac34;': '¾',
 '&iquest;': '¿',
 '&times;': '×',
 '&divide;': '÷',
 '&ETH;': 'Ð',
 '&eth;': 'ð',
 '&THORN;': 'Þ',
 '&thorn;': 'þ',
 '&AElig;': 'Æ',
 '&aelig;': 'æ',
 '&OElig;': 'Œ',
 '&oelig;': 'œ',
 '&Aring;': 'Å',
 '&Oslash;': 'Ø',
 '&Ccedil;': 'Ç',
 '&ccedil;': 'ç',
 '&szlig;': 'ß',
 '&Ntilde;': 'Ñ',
 '&ntilde;': 'ñ'}

special = {
    "’":"'",
    "‘":"'",
    "`":"'",
    '“':'"',
    '”':'"',
    '…':"."
}
        
def removeTags(text,splitter):
    div = text.split(splitter)
    endExists = True
    i = len(div)-1
    while i>=0 and endExists:
        if len(div[i].strip().split(" "))  == 1:
            div.pop(i)
            i-=1
        else:
            endExists = False
        
    return " "+splitter.join(div).strip()

def removeTagsFromStart(text,splitter):
    div = text.split(splitter)
    endExists = True
    i = 0
    while len(div)>0 and endExists:
        if len(div[i].strip().split(" "))  == 1:
            div=div[i+1:]
        else:
            div[i] = splitter+div[i]
            endExists = False
    if len(div) == 0:
        return ''
    splitfirst = div[0].split(" ")
    if "you" in splitfirst[1].lower() and "@" in splitfirst[0]:
        splitfirst[1] = splitfirst[0]
        splitfirst = splitfirst[1:]
        div[0] = " ".join(splitfirst)
    if "@" in splitfirst[0].strip()[0:2]:
        splitfirst = splitfirst[1:]
        div[0] = " ".join(splitfirst)
    return " ".join(div).strip()


def removeRT(text):
    if text[0:2] == 'RT':
        return ":".join(text.split(":")[1:]).strip()
    return text  

def isEnglish(s):
    try:
        return(detect(s) == 'en')
    except:
        return('')

def cleanTweet(text,removeFromMiddle):
    text = text.strip()
    for key,value in abbr_dict.items():
        text = re.sub(key,value,text)
    for key,value in extraChar.items():
        text = re.sub(key,value,text)
    for key,value in special.items():
        text = re.sub(key,value,text)
        #print(text)
    if removeFromMiddle:
        text = re.sub("@[A-Za-z0-9_]+","", text)
        text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub(r"http\S+", "", text)
    text = emoji_pattern.sub(r' ', text)
    text = removeTags(text,"#")
    text = removeTags(text,"@")
    text = removeTags(text,"#")
    text = removeTagsFromStart(text,"@")
    text = removeRT(text)
    text = re.sub(' +', ' ', text)
    text = re.sub("@",'',text)
    text = re.sub("#",'',text)
    text = re.sub(r'[\n\r]+',r'\n',text)
    text = re.sub('(?<![.?!])\n',". ",text)
    text = re.sub('\n'," ",text)
    #text = ' '.join(text.replace('\r', ' ').split())
    text = re.sub("\s+"," ",text)
    #text = re.sub(r"[^A-Za-z.!?'', ]",'',text)
    
    if not isEnglish(text):
        return ''
    return text.strip()


txt ='''Agree … Nature will do what nature does. Like humans trying to change the climate. #FoolsErrand @GavinNewsom admits face masks don’t work. They were required in all those places. And they did not stop the spread. And they won’t.  8% have 🦇 Flu: 6K of 3.2M in CA have died 0.014%'''
cleanTweet(txt,False)


"Agree . Nature will do what nature does. Like humans trying to change the climate. FoolsErrand GavinNewsom admits face masks don't work. They were required in all those places. And they did not stop the spread. And they won't. 8% have 🦇 Flu: 6K of 3.2M in CA have died 0.014%"

In [69]:
txt ='''The West Coast wildfires are what environmental injustice looks like. Only those who can afford air conditioners &amp; purifiers are able to escape the smoke.

This is why we won’t stop fighting for a just, livable world for everyone. Our health depends on it. https://t.co/z58ZpkUvJ2'''
#txt.replace(r'(?<![.?!])\n+',". ")
t = re.sub('[\n\r]+','\n',txt)
#re.sub('[\n\r]+',". ",txt)
t = re.sub('(?<![.?!])\n',". ",t)
re.sub('\n'," ",t)


'The West Coast wildfires are what environmental injustice looks like. Only those who can afford air conditioners &amp; purifiers are able to escape the smoke. This is why we won’t stop fighting for a just, livable world for everyone. Our health depends on it. https://t.co/z58ZpkUvJ2'

In [53]:
df['cleantxt'] = df['full_tweet_text'].apply(clean_tweet,False)

TypeError: cleanTweet() missing 1 required positional argument: 'removeFromMiddle'

In [56]:
df.iloc[2]['full_tweet_text']

'See this Instagram post of #cat #sleeping #heatwave by #me #thisisme @digital_moravian https://t.co/F7KvYMTknc #pets #heat #humidity #GlobalWarming #weather #summer #sucks'

In [60]:
df.iloc[2]['cleantxt']

'See this Instagram post of cat sleeping heatwave by'

In [14]:
txt = "@realDonaldTrump You &amp; don't know what you are doing?"
cleanTweet(txt,False)

'do not know what you are doing'

In [12]:
for key,value in extraChar.items():
        print(re.sub(key,value,txt))

@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You & don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump You &amp; don't know what you are doing?
@realDonaldTrump Y